In [1]:
import os 

import pandas as pd

DATA_DIR = "../data"
RESULTS_DIR = os.path.join(DATA_DIR, "results")

In [2]:
results_df = pd.concat([pd.read_csv(os.path.join(RESULTS_DIR, f)) for f in os.listdir(RESULTS_DIR) if f[-4:] == ".csv"], ignore_index=True)

In [3]:
grouped_results_df = pd.concat([
    results_df.groupby(["imputer", "Model", "attr_group", "target"])[["AUC", "Accuracy", "Sensitivity (Recall)", "Specificity"]].mean(),
    results_df.groupby(["imputer", "Model", "attr_group", "target"])[["n_positive", "n_total"]].sum()
], axis=1).reset_index().drop("imputer", axis=1)

In [4]:
grouped_results_df["target"] = grouped_results_df["target"].str.lstrip("Dia_")
grouped_results_df

,Model,attr_group,target,AUC,Accuracy,Sensitivity (Recall),Specificity,n_positive,n_total
0,RandomForest,MICE,HFD,0.734923,0.652494,0.712121,0.571307,6829.0,11266.0
1,RandomForest,MICE,HFPEF,0.584249,0.814176,0.039213,0.997710,1052.0,5489.0
2,RandomForest,MICE,HFREF,0.678867,0.824198,0.085071,0.993922,1018.0,5455.0
3,RandomForest,expert,HFD,0.905092,0.826291,0.826543,0.826249,6829.0,11266.0
4,RandomForest,expert,HFPEF,0.880225,0.884682,0.471682,0.982423,1052.0,5489.0
5,RandomForest,expert,HFREF,0.947804,0.927589,0.711468,0.977734,1018.0,5455.0
6,RandomForest,expert_blood,HFD,0.930265,0.860022,0.859649,0.860251,6829.0,11266.0
7,RandomForest,expert_blood,HFPEF,0.937770,0.912552,0.589837,0.988562,1052.0,5489.0
8,RandomForest,expert_blood,HFREF,0.983932,0.960953,0.837134,0.989888,1018.0,5455.0
9,RandomForest,secondary,HFD,0.947626,0.876656,0.882984,0.868739,5446.0,9591.0


In [5]:
best_models_df = grouped_results_df.groupby(["attr_group", "target"]).apply(lambda x: x.loc[x["AUC"].idxmax()])

C:\Users\flis1\AppData\Local\Temp\ipykernel_20904\1337778183.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_models_df = grouped_results_df.groupby(["attr_group", "target"]).apply(lambda x: x.loc[x["AUC"].idxmax()])


In [10]:
best_models_df

Model    attr_group target       AUC  Accuracy  \
attr_group   target                                                          
MICE         HFD     RandomForest          MICE    HFD  0.734923  0.652494   
             HFPEF        XGBoost          MICE  HFPEF  0.584875  0.814905   
             HFREF        XGBoost          MICE  HFREF  0.680890  0.824565   
expert       HFD     RandomForest        expert    HFD  0.905092  0.826291   
             HFPEF   RandomForest        expert  HFPEF  0.880225  0.884682   
             HFREF        XGBoost        expert  HFREF  0.948148  0.927039   
expert_blood HFD     RandomForest  expert_blood    HFD  0.930265  0.860022   
             HFPEF        XGBoost  expert_blood  HFPEF  0.941443  0.912006   
             HFREF        XGBoost  expert_blood  HFREF  0.985313  0.958753   
secondary    HFD     RandomForest     secondary    HFD  0.947626  0.876656   
             HFPEF   RandomForest     secondary  HFPEF  0.970569  0.923091   
             HFREF   RandomForest     secondary  HFREF  0.999492  0.994560   

                     Sensitivity (Recall)  Specificity  n_positive  n_total  
attr_group   target                                                          
MICE         HFD                 0.712121     0.571307      6829.0  11266.0  
             HFPEF               0.037451     0.999087      1052.0   5489.0  
             HFREF               0.086138     0.994158      1018.0   5455.0  
expert       HFD                 0.826543     0.826249      6829.0  11266.0  
             HFPEF               0.471682     0.982423      1052.0   5489.0  
             HFREF               0.715053     0.976154      1018.0   5455.0  
expert_blood HFD                 0.859649     0.860251      6829.0  11266.0  
             HFPEF               0.620710     0.980385      1052.0   5489.0  
             HFREF               0.850494     0.984032      1018.0   5455.0  
secondary    HFD                 0.882984     0.868739      5446.0   9591.0  
             HFPEF               0.735211     0.970609      1043.0   5188.0  
             HFREF               0.996995     0.993934      1001.0   5146.0

In [6]:
rename_cols = {"Sensitivity (Recall)": "Sensitivity", "AUC": "c-index", "target": "Predicting", "attr_group": "Attributes"}
final_selection = ["Attributes", "Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

In [7]:
compare_attr_df = best_models_df.loc[pd.IndexSlice[:, ["HFD"]], :].iloc[:-1].rename(columns=rename_cols)
compare_attr_df["Attributes"] = ["MICE", "Expert Selection", "Expert + Blood"]
compare_attr_df["Majority Accuracy"] = compare_attr_df["n_positive"]/ compare_attr_df["n_total"]
compare_attr_df.loc[:, compare_attr_df.dtypes.eq("float")] = compare_attr_df.loc[:, compare_attr_df.dtypes.eq("float")].round(3)
compare_attr_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_attrs.csv"), index=None)
compare_attr_df.loc[:, final_selection]

,,Attributes,Predicting,Model,c-index,Accuracy,Majority Accuracy,Sensitivity,Specificity
attr_group,target,,,,,,,,
MICE,HFD,MICE,HFD,RandomForest,0.735,0.652,0.606,0.712,0.571
expert,HFD,Expert Selection,HFD,RandomForest,0.905,0.826,0.606,0.827,0.826
expert_blood,HFD,Expert + Blood,HFD,RandomForest,0.930,0.860,0.606,0.860,0.860


In [8]:
final_selection = ["Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

compare_expert_df = best_models_df.loc[pd.IndexSlice["expert", :], :].rename(columns=rename_cols)
compare_expert_df["Majority Accuracy"] = compare_expert_df["n_positive"]/ compare_expert_df["n_total"]
compare_expert_df["Majority Accuracy"] = compare_expert_df["Majority Accuracy"].where(
    compare_expert_df["Majority Accuracy"].gt(0.5), 1-compare_expert_df["Majority Accuracy"]
)
compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")] = compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")].round(3)

compare_expert_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_expert.csv"), index=None)
compare_expert_df.loc[:, final_selection]

Predicting         Model  c-index  Accuracy  \
attr_group target                                               
expert     HFD           HFD  RandomForest    0.905     0.826   
           HFPEF       HFPEF  RandomForest    0.880     0.885   
           HFREF       HFREF       XGBoost    0.948     0.927   

                   Majority Accuracy  Sensitivity  Specificity  
attr_group target                                               
expert     HFD                 0.606        0.827        0.826  
           HFPEF               0.808        0.472        0.982  
           HFREF               0.813        0.715        0.976

In [9]:
final_selection = ["Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

compare_expert_df = best_models_df.loc[pd.IndexSlice["expert_blood", :], :].rename(columns=rename_cols)
compare_expert_df["Majority Accuracy"] = compare_expert_df["n_positive"]/ compare_expert_df["n_total"]
compare_expert_df["Majority Accuracy"] = compare_expert_df["Majority Accuracy"].where(
    compare_expert_df["Majority Accuracy"].gt(0.5), 1-compare_expert_df["Majority Accuracy"]
)
compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")] = compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")].round(3)

compare_expert_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_expert_blood.csv"), index=None)
compare_expert_df.loc[:, final_selection]

Predicting         Model  c-index  Accuracy  \
attr_group   target                                               
expert_blood HFD           HFD  RandomForest    0.930     0.860   
             HFPEF       HFPEF       XGBoost    0.941     0.912   
             HFREF       HFREF       XGBoost    0.985     0.959   

                     Majority Accuracy  Sensitivity  Specificity  
attr_group   target                                               
expert_blood HFD                 0.606        0.860        0.860  
             HFPEF               0.808        0.621        0.980  
             HFREF               0.813        0.850        0.984

In [12]:
final_selection = ["Predicting", "Model", "c-index", "Accuracy", "Majority Accuracy", "Sensitivity", "Specificity"]

compare_expert_df = best_models_df.loc[[pd.IndexSlice["secondary", "HFD"]], :].rename(columns=rename_cols)
compare_expert_df["Majority Accuracy"] = compare_expert_df["n_positive"]/ compare_expert_df["n_total"]
compare_expert_df["Majority Accuracy"] = compare_expert_df["Majority Accuracy"].where(
    compare_expert_df["Majority Accuracy"].gt(0.5), 1-compare_expert_df["Majority Accuracy"]
)
compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")] = compare_expert_df.loc[:, compare_expert_df.dtypes.eq("float")].round(3)

compare_expert_df.loc[:, final_selection].to_csv(os.path.join(RESULTS_DIR, "final_tables", "compare_secondary.csv"), index=None)
compare_expert_df.loc[:, final_selection]

,,Predicting,Model,c-index,Accuracy,Majority Accuracy,Sensitivity,Specificity
attr_group,target,,,,,,,
secondary,HFD,HFD,RandomForest,0.948,0.877,0.568,0.883,0.869
